In [1]:
# -------------------------------
# Step 0: Imports
# -------------------------------
import os
import scanpy as sc
import pandas as pd
import gzip
import shlex
import re
import time
import numpy as np
from scipy import sparse
from tqdm.auto import tqdm

In [2]:
# -------------------------------
# Step 1: Define paths + output directory
# -------------------------------
counts_file = "GSE267033_counts_RNAseq_sc_WBC.txt.gz"
meta_file   = "GSE267033_metadata_RNAseq_sc_WBC.txt.gz"

out_dir = "GSE267033_h5ad"  # renamed to match later code usage
os.makedirs(out_dir, exist_ok=True)

In [3]:
# -------------------------------
# Step 2: Parse metadata (unchanged)
# -------------------------------
with gzip.open(meta_file, "rt") as f:
    header = shlex.split(f.readline().strip())  # 11 names (no cell id name)

    rows = []
    for line in f:
        line = line.strip()
        if not line:
            continue
        rows.append(shlex.split(line))          # 12 fields (cell_id + 11 columns)

# If rows have one extra field vs header, the extra is the cell id
if rows and len(rows[0]) == len(header) + 1:
    header = ["cell_id"] + header

meta = pd.DataFrame(rows, columns=header).set_index("cell_id")

print(meta.shape)
print(meta.columns.tolist())
print("Cell IDs example:", meta.index[:5].tolist())

(29890, 11)
['n_genes', 'n_counts', 'pct_counts_mitochondrial', 'pct_counts_ribosomal', 'gender', 'leiden', 'cell.type', 'NAS_group', 'NAS', 'Control_MAFLD', 'sampleID']
Cell IDs example: ['AGCGTCGTCCGGTAAT-1_17_17-White_blood_cells_patient_15', 'TTGGGCGAGACAAGCC-1_17_17-White_blood_cells_patient_15', 'TGTGCGGTCTCGTCGT-1_17_17-White_blood_cells_patient_15', 'CGGAACCTCCTAGCCT-1_17_17-White_blood_cells_patient_15', 'ATTGTTCTCCGTACGG-1_17_17-White_blood_cells_patient_15']


In [4]:
# -------------------------------
# Step 3: Counts parsing (header cell IDs) (kept)
# -------------------------------
def split_first_token(line: str):
    line = line.strip()
    if not line:
        return None, None
    if line[0] == '"':
        j = line.find('"', 1)
        if j == -1:
            raise ValueError("Malformed quoted token.")
        token = line[1:j]
        rest = line[j+1:].strip()
        return token, rest
    parts = line.split(maxsplit=1)
    token = parts[0]
    rest = parts[1] if len(parts) > 1 else ""
    return token, rest

t0 = time.time()

with gzip.open(counts_file, "rt") as f:
    header_line = f.readline().strip()
    cell_ids = re.findall(r'"([^"]+)"', header_line)
    n_cells = len(cell_ids)
    if n_cells == 0:
        raise ValueError("Failed to parse cell IDs from header (expected quoted cell IDs).")

    gene_names = []
    indptr = [0]
    indices = []
    data = []

    # Unknown total rows in gz stream -> tqdm without total
    for line_num, line in enumerate(tqdm(f, desc="Reading genes", unit="genes"), start=2):
        gene, rest = split_first_token(line)
        if gene is None:
            continue

        vals = np.fromstring(rest, sep=" ")
        if vals.size != n_cells:
            raise ValueError(
                f"Line {line_num}: expected {n_cells} counts, got {vals.size}."
            )

        nz = np.nonzero(vals)[0]
        if nz.size:
            indices.extend(nz.tolist())
            data.extend(vals[nz].astype(np.float32).tolist())

        indptr.append(len(indices))
        gene_names.append(gene)

elapsed = time.time() - t0
print(f"Parsed genes={len(gene_names):,}, cells={n_cells:,}, nnz={len(indices):,} in {elapsed/60:.1f} min")

Reading genes: 0genes [00:00, ?genes/s]

Parsed genes=58,051, cells=29,890, nnz=53,955,730 in 2.9 min


In [5]:
# -------------------------------
# Step 4: Create sparse matrix + AnnData (unchanged)
# -------------------------------
X_gxc = sparse.csr_matrix(
    (np.array(data, dtype=np.float32),
     np.array(indices, dtype=np.int32),
     np.array(indptr, dtype=np.int64)),
    shape=(len(gene_names), n_cells),
)

adata = sc.AnnData(X_gxc.T)
adata.obs_names = cell_ids
adata.var_names = gene_names

print("adata shape:", adata.shape, "nnz:", adata.X.nnz)

adata shape: (29890, 58051) nnz: 53955730


In [6]:
# -------------------------------
# Step 5: Metadata overlap check (~1.0)
# -------------------------------
overlap = adata.obs_names.isin(meta.index).mean()
print("Fraction of cells with metadata:", overlap)

Fraction of cells with metadata: 1.0


In [7]:
# -------------------------------
# Step 6: Attach metadata to adata.obs (remove redundancy)
# -------------------------------
# Lenient: preserves order, inserts NaN for any missing metadata rows
adata.obs = meta.reindex(adata.obs_names).copy()

In [8]:
# -------------------------------
# Step 7: Targeted numeric conversion (kept)
# -------------------------------
num_cols = [
    "n_genes",
    "n_counts",
    "pct_counts_mitochondrial",
    "pct_counts_ribosomal",
    "leiden",
    "NAS",
]
for c in num_cols:
    if c in adata.obs.columns:
        adata.obs[c] = pd.to_numeric(adata.obs[c], errors="coerce")

# Optional: add dataset label
adata.obs["dataset"] = "GSE267033"

In [9]:
# -------------------------------
# Step 8: Save combined h5ad (correct out_path to use out_dir)
# -------------------------------
out_path = os.path.join(out_dir, "GSE267033_WBC.h5ad")
adata.write(out_path)
print("Saved:", out_path)

Saved: GSE267033_h5ad/GSE267033_WBC.h5ad


In [10]:
# -------------------------------
# Step 9: Quick checks (unchanged structure; keep/remove as desired)
# -------------------------------
print(adata)
print(adata.obs.head())                  # Checking header
print(adata.var_names[:5].tolist())      # Checking first 5 genes
print("Any missing metadata rows?", adata.obs.isna().any(axis=1).mean())

AnnData object with n_obs × n_vars = 29890 × 58051
    obs: 'n_genes', 'n_counts', 'pct_counts_mitochondrial', 'pct_counts_ribosomal', 'gender', 'leiden', 'cell.type', 'NAS_group', 'NAS', 'Control_MAFLD', 'sampleID', 'dataset'
                                                    n_genes  n_counts  \
AGCGTCGTCCGGTAAT-1_17_17-White_blood_cells_pati...      743      1217   
TTGGGCGAGACAAGCC-1_17_17-White_blood_cells_pati...      206       289   
TGTGCGGTCTCGTCGT-1_17_17-White_blood_cells_pati...     1973      4529   
CGGAACCTCCTAGCCT-1_17_17-White_blood_cells_pati...      210       305   
ATTGTTCTCCGTACGG-1_17_17-White_blood_cells_pati...      774      1619   

                                                    pct_counts_mitochondrial  \
AGCGTCGTCCGGTAAT-1_17_17-White_blood_cells_pati...                  5.669680   
TTGGGCGAGACAAGCC-1_17_17-White_blood_cells_pati...                 19.377163   
TGTGCGGTCTCGTCGT-1_17_17-White_blood_cells_pati...                 10.885405   
CGGAACCTCCTAGC

In [11]:
# -------------------------------
# Step 10: Per-patient exports (kept)
# -------------------------------
def safe_name(s: str) -> str:
    return re.sub(r"[^A-Za-z0-9._-]+", "_", str(s))

sample_series = adata.obs["sampleID"].astype(str)

per_patient_paths = []

for patient_id in sample_series.unique():
    if patient_id in {"NA", "nan", "None", "<NA>"}:
        continue

    mask = (sample_series == patient_id).to_numpy(dtype=bool)
    ad_p = adata[mask, :].copy()

    path = os.path.join(out_dir, f"GSE267033_{safe_name(patient_id)}.h5ad")
    ad_p.write(path)
    per_patient_paths.append(path)
    print(f"{patient_id}: {ad_p.n_obs} cells -> {path}")

White_blood_cells_patient_15: 3252 cells -> GSE267033_h5ad/GSE267033_White_blood_cells_patient_15.h5ad
White_blood_cells_patient_2: 1767 cells -> GSE267033_h5ad/GSE267033_White_blood_cells_patient_2.h5ad
White_blood_cells_patient_5: 1479 cells -> GSE267033_h5ad/GSE267033_White_blood_cells_patient_5.h5ad
White_blood_cells_patient_6: 2257 cells -> GSE267033_h5ad/GSE267033_White_blood_cells_patient_6.h5ad
White_blood_cells_patient_8: 1284 cells -> GSE267033_h5ad/GSE267033_White_blood_cells_patient_8.h5ad
White_blood_cells_patient_10: 2815 cells -> GSE267033_h5ad/GSE267033_White_blood_cells_patient_10.h5ad
White_blood_cells_control_1: 3252 cells -> GSE267033_h5ad/GSE267033_White_blood_cells_control_1.h5ad
White_blood_cells_control_2: 2208 cells -> GSE267033_h5ad/GSE267033_White_blood_cells_control_2.h5ad
White_blood_cells_control_3: 1468 cells -> GSE267033_h5ad/GSE267033_White_blood_cells_control_3.h5ad
White_blood_cells_control_4: 2058 cells -> GSE267033_h5ad/GSE267033_White_blood_cells_c

In [12]:
# -------------------------------
# Step 11: Verify per-patient files by re-reading and counting observations
# -------------------------------
check_rows = []
for path in per_patient_paths:
    ad_chk = sc.read_h5ad(path)
    check_rows.append((os.path.basename(path), ad_chk.n_obs))

check_df = pd.DataFrame(check_rows, columns=["file", "n_obs"]).sort_values("file")
print(check_df.to_string(index=False))
print("Total n_obs across per-patient files:", int(check_df["n_obs"].sum()))
print("Original adata n_obs:", int(adata.n_obs))

                                       file  n_obs
 GSE267033_White_blood_cells_control_1.h5ad   3252
 GSE267033_White_blood_cells_control_2.h5ad   2208
 GSE267033_White_blood_cells_control_3.h5ad   1468
 GSE267033_White_blood_cells_control_4.h5ad   2058
 GSE267033_White_blood_cells_control_5.h5ad   1401
 GSE267033_White_blood_cells_control_9.h5ad   1984
GSE267033_White_blood_cells_patient_10.h5ad   2815
GSE267033_White_blood_cells_patient_13.h5ad   1584
GSE267033_White_blood_cells_patient_14.h5ad   2515
GSE267033_White_blood_cells_patient_15.h5ad   3252
 GSE267033_White_blood_cells_patient_2.h5ad   1767
GSE267033_White_blood_cells_patient_36.h5ad    566
 GSE267033_White_blood_cells_patient_5.h5ad   1479
 GSE267033_White_blood_cells_patient_6.h5ad   2257
 GSE267033_White_blood_cells_patient_8.h5ad   1284
Total n_obs across per-patient files: 29890
Original adata n_obs: 29890


In [13]:
### Copy to new dir and rename

In [16]:
# -------------------------------
# Step 12: Imports
# -------------------------------
import os
import re
import shutil
from glob import glob
from typing import Optional

In [17]:
# -------------------------------
# Step 13: Define source + destination
# -------------------------------
src_dir = "/home/tsaliu/GSE267033_h5ad"
dst_dir = "/home/tsaliu/WBC Samples"  # new folder (contains a space)
os.makedirs(dst_dir, exist_ok=True)

In [18]:
# -------------------------------
# Step 14: Helper to build new filename
#   GSE267033_White_blood_cells_control_1.h5ad  -> WBC_CT-1.h5ad
#   GSE267033_White_blood_cells_patient_10.h5ad -> WBC_PT-10.h5ad
# -------------------------------
def make_new_name(old_name: str) -> Optional[str]:
    """
    Returns the new filename (not full path), or None if it doesn't match expected pattern.
    """
    base = os.path.basename(old_name)

    m = re.match(
        r"^GSE267033_White_blood_cells_(control|patient)_(\d+)\.h5ad$",
        base
    )
    if not m:
        return None

    group = m.group(1)  # control or patient
    num = m.group(2)    # digits
    tag = "CT" if group == "control" else "PT"
    return f"WBC_{tag}-{num}.h5ad"

In [19]:
# -------------------------------
# Step 15: Copy + rename
# -------------------------------
src_files = sorted(glob(os.path.join(src_dir, "*.h5ad")))
copied = 0
skipped = []

for src_path in src_files:
    new_name = make_new_name(src_path)
    if new_name is None:
        skipped.append(os.path.basename(src_path))
        continue

    dst_path = os.path.join(dst_dir, new_name)

    # Copy file metadata too
    shutil.copy2(src_path, dst_path)
    copied += 1
    print(f"Copied: {os.path.basename(src_path)} -> {dst_path}")

print(f"\nDone. Copied {copied} files to: {dst_dir}")
if skipped:
    print("Skipped (did not match pattern):")
    for s in skipped:
        print("  -", s)

Copied: GSE267033_White_blood_cells_control_1.h5ad -> /home/tsaliu/WBC Samples/WBC_CT-1.h5ad
Copied: GSE267033_White_blood_cells_control_2.h5ad -> /home/tsaliu/WBC Samples/WBC_CT-2.h5ad
Copied: GSE267033_White_blood_cells_control_3.h5ad -> /home/tsaliu/WBC Samples/WBC_CT-3.h5ad
Copied: GSE267033_White_blood_cells_control_4.h5ad -> /home/tsaliu/WBC Samples/WBC_CT-4.h5ad
Copied: GSE267033_White_blood_cells_control_5.h5ad -> /home/tsaliu/WBC Samples/WBC_CT-5.h5ad
Copied: GSE267033_White_blood_cells_control_9.h5ad -> /home/tsaliu/WBC Samples/WBC_CT-9.h5ad
Copied: GSE267033_White_blood_cells_patient_10.h5ad -> /home/tsaliu/WBC Samples/WBC_PT-10.h5ad
Copied: GSE267033_White_blood_cells_patient_13.h5ad -> /home/tsaliu/WBC Samples/WBC_PT-13.h5ad
Copied: GSE267033_White_blood_cells_patient_14.h5ad -> /home/tsaliu/WBC Samples/WBC_PT-14.h5ad
Copied: GSE267033_White_blood_cells_patient_15.h5ad -> /home/tsaliu/WBC Samples/WBC_PT-15.h5ad
Copied: GSE267033_White_blood_cells_patient_2.h5ad -> /home/ts

In [20]:
# -------------------------------
# Step 16: Verify destination filenames + counts
# -------------------------------
dst_files = sorted(glob(os.path.join(dst_dir, "*.h5ad")))
print(f"\nDestination file count: {len(dst_files)}")
print("Examples:")
for p in dst_files[:10]:
    print("  -", os.path.basename(p))


Destination file count: 15
Examples:
  - WBC_CT-1.h5ad
  - WBC_CT-2.h5ad
  - WBC_CT-3.h5ad
  - WBC_CT-4.h5ad
  - WBC_CT-5.h5ad
  - WBC_CT-9.h5ad
  - WBC_PT-10.h5ad
  - WBC_PT-13.h5ad
  - WBC_PT-14.h5ad
  - WBC_PT-15.h5ad
